In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model
import os, shutil, random


2025-12-15 10:03:57.280484: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-15 10:03:57.573498: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 10:04:18.565571: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-15 10:04:32.410413: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

In [2]:
import os
import shutil
import random

# Carpeta donde están TODAS las imágenes mezcladas
data_dir = "..data/dogs-vs-cats"

# Carpetas destino
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")

train_cats = os.path.join(train_dir, "cats")
train_dogs = os.path.join(train_dir, "dogs")
test_cats  = os.path.join(test_dir, "cats")
test_dogs  = os.path.join(test_dir, "dogs")

dest_folders = [train_cats, train_dogs, test_cats, test_dogs]


# --------------------------------------------------------------------
# 2. CREAR CARPETAS LIMPIAS
# --------------------------------------------------------------------
for folder in dest_folders:
    os.makedirs(folder, exist_ok=True)
    print(f"📁 Creada carpeta: {folder}")

# --------------------------------------------------------------------
# 3. OBTENER LISTA DE IMÁGENES
# --------------------------------------------------------------------
all_images = [img for img in os.listdir(data_dir) if img.lower().endswith(".jpg")]

cat_images = [img for img in all_images if img.lower().startswith("cat")]
dog_images = [img for img in all_images if img.lower().startswith("dog")]

print(f"\nTotal gatos al principio: {len(cat_images)}")
print(f"Total perros: {len(dog_images)}\n")

# --------------------------------------------------------------------
#  4. FUNCIÓN PARA SPLIT 80/20 Y COPIAR IMÁGENES
# --------------------------------------------------------------------
def split_and_copy(images, train_path, test_path, train_ratio=0.8):
    random.shuffle(images)
    split_idx = int(len(images) * train_ratio)

    train_imgs = images[:split_idx]
    test_imgs  = images[split_idx:]

    for img in train_imgs:
        shutil.move(os.path.join(data_dir, img), train_path)

    for img in test_imgs:
        shutil.move(os.path.join(data_dir, img), test_path)

    print(f"{train_path}: {len(train_imgs)} train")
    print(f"{test_path}:  {len(test_imgs)} test\n")

# --------------------------------------------------------------------
#  5. SPLIT PARA GATOS Y PERROS
# --------------------------------------------------------------------
split_and_copy(cat_images, train_cats, test_cats)
split_and_copy(dog_images, train_dogs, test_dogs)

📁 Creada carpeta: ..data/dogs-vs-cats/train/cats
📁 Creada carpeta: ..data/dogs-vs-cats/train/dogs
📁 Creada carpeta: ..data/dogs-vs-cats/test/cats
📁 Creada carpeta: ..data/dogs-vs-cats/test/dogs

Total gatos al principio: 0
Total perros: 0

..data/dogs-vs-cats/train/cats: 0 train
..data/dogs-vs-cats/test/cats:  0 test

..data/dogs-vs-cats/train/dogs: 0 train
..data/dogs-vs-cats/test/dogs:  0 test



In [3]:
SOURCE_DIR = "../data/dogs-vs-cats/train"

# Carpetas destino
CAT_DIR = os.path.join(SOURCE_DIR, "cats")
DOG_DIR = os.path.join(SOURCE_DIR, "dogs")

# Crear carpetas
os.makedirs(CAT_DIR, exist_ok=True)
os.makedirs(DOG_DIR, exist_ok=True)

# Mover imágenes según el nombre
for fname in os.listdir(SOURCE_DIR):
    fpath = os.path.join(SOURCE_DIR, fname)

    # Ignorar directorios
    if os.path.isdir(fpath):
        continue
    
    # Solo procesar imágenes
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    # Clasificar según nombre
    if "cat" in fname.lower():
        shutil.move(fpath, os.path.join(CAT_DIR, fname))
    elif "dog" in fname.lower():
        shutil.move(fpath, os.path.join(DOG_DIR, fname))

In [4]:
orig_dir = "../data/dogs-vs-cats/train"
base_dir = "../dogs-vs-cats" 


for split in ["train_split", "val_split", "test_split"]:
    for cls in ["cats", "dogs"]:
        os.makedirs(os.path.join(base_dir, split, cls), exist_ok=True)

# Listar imágenes correctamente
cats = [f for f in os.listdir(os.path.join(orig_dir, "cats")) if f.endswith(".jpg")]
dogs = [f for f in os.listdir(os.path.join(orig_dir, "dogs")) if f.endswith(".jpg")]

def move_split(files, cls):
    random.shuffle(files)
    n = len(files)
    n_train = int(0.7 * n)
    n_val   = int(0.15 * n)

    for i, f in enumerate(files):
        if i < n_train:
            dest = "train_split"
        elif i < n_train + n_val:
            dest = "val_split"
        else:
            dest = "test_split"

        shutil.copy(
            os.path.join(orig_dir, cls, f),
            os.path.join(base_dir, dest, cls, f)
        )

move_split(cats, "cats")
move_split(dogs, "dogs")

In [5]:
base_dir = "../data/dogs-vs-cats/train"

dog_dir = os.path.join(base_dir, "dogs")
cat_dir = os.path.join(base_dir, "cats")

# Comprobación (muy útil)
print("Dog dir exists:", os.path.isdir(dog_dir))
print("Cat dir exists:", os.path.isdir(cat_dir))
print("Ejemplo dogs:", os.listdir(dog_dir)[:5])
print("Ejemplo cats:", os.listdir(cat_dir)[:5])

# Obtener los primeros 9 archivos
dog_files = sorted(os.listdir(dog_dir))[:9]
cat_files = sorted(os.listdir(cat_dir))[:9]

# 9 perros
plt.figure(figsize=(8, 8))
for i, fname in enumerate(dog_files):
    img = load_img(os.path.join(dog_dir, fname))
    plt.subplot(3, 3, i+1)
    plt.imshow(img)
    plt.axis("off")
plt.suptitle("Primeras 9 imágenes de perros")
plt.show()


Dog dir exists: True
Cat dir exists: True
Ejemplo dogs: []
Ejemplo cats: []


<Figure size 800x800 with 0 Axes>

In [6]:

# 9 gatos
plt.figure(figsize=(8, 8))
for i, fname in enumerate(cat_files):
    img = load_img(os.path.join(cat_dir, fname))
    plt.subplot(3, 3, i+1)
    plt.imshow(img)
    plt.axis("off")
plt.suptitle("Primeras 9 imágenes de gatos")
plt.show()


<Figure size 800x800 with 0 Axes>

In [7]:
base_dir = "../data/dogs-vs-cats"

IMG_SIZE = (200, 200)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(os.path.join(base_dir, "train"),
                                              target_size=IMG_SIZE,
                                              batch_size=BATCH_SIZE,
                                              class_mode="binary")


test_gen = test_datagen.flow_from_directory(os.path.join(base_dir, "test"),
                                            target_size=IMG_SIZE,
                                            batch_size=BATCH_SIZE,
                                            class_mode="binary",
                                            shuffle=False)

Found 0 images belonging to 2 classes.


FileNotFoundError: [Errno 2] No such file or directory: '../data/dogs-vs-cats/test'